# Text Chunking for Noun Phrase

In [1]:

# Needed for Noun Phrase work-around
from io import StringIO  # from StringIO import cStringIO  #Python2
import sys
import json

from nltk.tokenize import sent_tokenize
import nltk.chunk
from nltk.tag import pos_tag
from nltk.draw.tree import TreeSegmentWidget, tree_to_treesegment

import itertools
import re

import numpy as np
import pandas as pd


In [ ]:
## Read 'PublicRemarks' Text from file
textfile = open('PublicRemarksCorpus.txt', 'r') # File
text = textfile.read()  # String
#text = text[0:493]

In [ ]:
## Convert Corpus to list of sentences
df = pd.Series(sent_tokenize(text))
#print(len(sent_tokenize_list))
#sent_tokenize_list

In [ ]:
## Clean Text - Runs slow and isn't parallelized, try pypi and/or multiprocessing
df = df.apply(lambda x : re.sub(r'[\b\(\)\\\"\'\/\[\]\s+\,\.:\?!;]', ' ', x))
df
df.describe()

In [ ]:
## Gather noun phrase (NP) from sentence tree
class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        sys.stdout = self._stdout

def traverse(t):
    try:
        t.label()
    except AttributeError:
        return
    else:
        if t.label() == 'NP': print(t)  # or do something else
        else:
            for child in t: 
                traverse(child)

def nounPhrase(sentence):
    # Tag sentence for part of speech
    tagged_sent = pos_tag(sentence.split())  # List of tuples with [(Word, PartOfSpeech)]
    # Define several tag patterns
    grammar = r"""
      NP: {<DT|PP\$>?<JJ>*<NN>}   # chunk determiner/possessive, adjectives and noun
          {<NNP>+}                # chunk sequences of proper nouns
          {<NN>+}                 # chunk consecutive nouns
          {<CD>+}                 # chunk cardinal numbers i.e. Zip Codes
          {<JJ>+}                 # chunk hyphenated terms
          """

    cp = nltk.RegexpParser(grammar)
    SentenceTree = cp.parse(tagged_sent)
    #NounPhrases = traverse(SentenceTree)

    with Capturing() as output:  # work-around for recursive traverse statement
        traverse(SentenceTree)   # returns a variable 'output'
    
    #NounPhrases = output
    #print(NounPhrases)
    return(output)

df2 = df.apply(lambda x : nounPhrase(x))
df2 = list(df2)
df2

In [ ]:
## Clean up Noun Phrases 
## Maybe later...
#df2 = df2.apply(lambda x : x.str.replace('(NP ', '') )
#df2 = df2.replace('/NN', '')

print(df2[0])

In [ ]:
## dictionary
chain = np.concatenate(df2)
#chain = itertools.chain.from_iterable(df2)
dictionary = list(chain)
#print(dictionary)

In [ ]:
print(len(dictionary)) # 2.4 M

In [ ]:
## Remove Duplicates (order & count are lost!)
dictionary = set(dictionary)
' #1/NNP' in dictionary
#dict = list(dictionary)
#len(dict)  # 209 k

In [ ]:
## Save Dictionary to JSON
with open('dict.json', 'w') as outfile:
    json.dump(dict, outfile)

In [ ]:
# Read Dictionary from file
f = open('dict.json','r')
data = f.read()
dict = json.loads(data)
dict[0:10]

In [ ]:
print(sorted(Dictionary)[0:100]) # print first strings in list

In [ ]:
## Replace Text to clean list
dict = set(Dictionary)  # Use to create the Attribute set
print(sorted(dict)[0:100])

In [ ]:
'(NP lease/NN)' in dict

In [ ]:
## Unigram, Bigram, Trigram

In [ ]:
## Deprecated
## Test Sentences for Debugging
#text = 'Have you been to the fixer-upper in the 84505 6208 area code?'
#text = 'We saw the yellow dog.'
text = 'Robert Johnson, the cat in the hat, shot his giant cannon!!!' #Why did he do that?'
#text = "Michael Jackson likes to eat at McDonalds." 

In [ ]:
## Deprecated!
## Clean Text
# Remove punctuation and set to lower case
text = re.sub(r'[\b\(\)\\\"\'\/\[\]\s+\,\.:\?!;]', ' ', text)
# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
sentence = '\n'.join(chunk for chunk in chunks if chunk)
sentence

In [ ]:
## Deprecated!
sentence
pos_tag(sentence.split())

In [ ]:
## Deprecated
# Tag sentence for part of speech
tagged_sent = pos_tag(sentence.split())  # List of tuples with [(Word, PartOfSpeech)]
# Define several tag patterns
grammar = r"""
      NP: {<DT|PP\$>?<JJ>*<NN>}   # chunk determiner/possessive, adjectives and noun
          {<NNP>+}                # chunk sequences of proper nouns
          {<NN>+}                 # chunk consecutive nouns
          {<CD>+}                 # chunk cardinal numbers i.e. Zip Codes
          {<JJ>+}                 # chunk hyphenated terms
          """

#cp = nltk.RegexpParser(grammar)
#SentenceTree = cp.parse(tagged_sent)
    #NounPhrases = traverse(SentenceTree)
    #print('')
    #print(NounPhrases)



#with Capturing() as NounPhrases:  # work-around for recursive traverse statement
cp = nltk.RegexpParser(grammar)       # configure parser
chunked_sent = cp.parse(tagged_sent)  # Tree Object
traverse(chunked_sent)  #  NounPhrases = 
    
#print(NounPhrases)

In [ ]:
print(type(SentenceTree))
print(SentenceTree)
print(NounPhrases)


In [ ]:
for subtree in chunked_sent.subtrees():
    if subtree.label() == 'NP': print(subtree)

In [ ]:
SentenceTree.draw()



In [ ]:
def traverseTree(tree):
    #print("tree:", tree)
    set = []
    for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
            traverseTree(subtree)
        else:
            if tree.label() == 'NP':
                set += tree[0][0]  #print(tree[0][0])
    return set

cp = nltk.RegexpParser(grammar)
SentenceTree = cp.parse(tagged_sent)
#print(SentenceTree)

NounPhrases2 = traverseTree(SentenceTree)
print(NounPhrases2)

#SentenceTree.leaves()




In [ ]:
%matplotlib inline

#import urllib
#from bs4 import BeautifulSoup
import nltk
#from nltk.tokenize import word_tokenize
#from nltk.tokenize import RegexpTokenizer
#from nltk.collocations import *
#import string

In [ ]:
# Python code for segmentation, POS tagging and tokenization
rawtext = open('PublicRemarksCorpus.txt').read()
sentences = nltk.sent_tokenize(rawtext) # NLTK default sentence segmenter 
sentences = [nltk.word_tokenize(sent) for sent in sentences] # NLTK word tokenizer
sentences = [nltk.pos_tag(sent) for sent in sentences] # NLTK POS tagger

In [ ]:
Patterns = """
        NP:     {<DT|PP\$>?<JJ>*<NN>}
                {<NNP>+}
                {<NN>+}
"""

NPChunker = nltk.RegexpParser(patterns) 

In [ ]:


url = 'https://en.wikipedia.org/wiki/N-gram'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html)

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

#textFiltered = filter(None,[word.strip(string.punctuation)
#                 for word in text.replace(';','; ').split()
#                 ])

tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(text)

finder = TrigramCollocationFinder.from_words(tokens)
trigram_measures = nltk.collocations.TrigramAssocMeasures()
scored = finder.score_ngrams(trigram_measures.raw_freq)
sorted(finder.nbest(trigram_measures.raw_freq, 10))  # Return 10 trigrams

### Create a corpus and conduct analysis

In [ ]:
## Concatenate 'PublicRemarks' into a corpus
def concatCol(col):
    text = ''  # Declare text as string
    for n in range(len(col)): # 
        text += str(col[n])+' '
    return text

corpus = concatCol(df['PublicRemarks'])

## Save Text to File
file = open('PublicRemarksCorpus.txt', "w")
file.write(corpus)
file.close()

In [ ]:
# Read 'PublicRemarks' Text from file
textfile = open('PublicRemarksCorpus.txt', 'r') # File
corpus = textfile.read()  # String
#corpus[0:500]

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def get_ngrams(text, n ):
    text = re.sub(r'[\b\(\)\\\"\'\/\[\]\s+\,\.:\?;]', ' ', text)
    text = text.lower()
    tokens = word_tokenize(text)
    n_grams = ngrams(tokens, n)
    return [ ' '.join(grams) for grams in n_grams]

unigrams = get_ngrams(corpus, 1)